In [29]:
from langchain_community.vectorstores import Chroma, Milvus
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import os
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from collections import defaultdict
from dotenv import load_dotenv
import re
from unstructured.partition.pdf import partition_pdf
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
import concurrent.futures
import pickle
from langchain_chroma import Chroma

from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
import json
from langchain.chains import LLMChain
from langchain_ollama import ChatOllama
from flashrank import Ranker, RerankRequest
import pandas as pd
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### **Ingestion**

In [2]:
data_path = ["./data_3/Doc_1.pdf",
            "./data_3/Doc_2.pdf",
            "./data_3/Doc_3.pdf",
            "./data_3/Doc_4.pdf",
            "./data_3/Doc_5.pdf"]

In [3]:
def get_documents(file_name):
    """loads pdf file as a document.
    file_name is path of file stored locally"""

    print(f"Extraction started for: {file_name} ...")

    elements = partition_pdf(file_name,
                             include_metadata=False,
                             strategy='hi_res')
    titles = []
    final_dict = {}
    filtered_data = {}

    title_text = defaultdict(list)

    for index, tag in enumerate(elements):  # putting all text under a title under it.

        if "Title" in str(type(tag)):
            titles.append(str(tag))
            for tags in elements[index + 1:]:
                if "Title" not in str(type(tags)) and "Footer" not in str(type(tags)):  # removing footer data
                    title_text[str(tag)].extend([str(tags)])
                else:
                    break

    for key, value in title_text.items():
        if value:
            final_dict[key] = value

    for key, value in final_dict.items():  # removing content and index data as it doesn't add value
        if 'content' in key.lower() or 'index' in key.lower() or len("".join(value)) <= 30:
            pass
        else:
            text = "".join(value)

            formatted_page_content = re.sub('…', '', text)
            formatted_key = re.sub('…', '', key)
            filtered_data[formatted_key] = formatted_page_content

    all_docs = []

    for key, value in filtered_data.items():
        all_docs.append(Document(page_content=key + ": " + value, metadata={"source": file_name.split("/")[-1], "title": key}))

    print(f"Extraction done for: {file_name} ...")

    return all_docs

In [4]:
def get_huggingface_embedding():
    encode_kwargs = {'normalize_embeddings': True}
    embedding_function = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5',
                                                encode_kwargs=encode_kwargs)
    return embedding_function

In [5]:
embedding_function = get_huggingface_embedding()

/anaconda/envs/azureml_py38_PT_TF/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2025-01-16 08:33:43.643955: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 08:33:47.605959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-16 08:33:47.606108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already 

In [11]:
def init_retriever(docs,embedding_function):
    """Init and Persists the chroma vector store"""

    db = Chroma.from_documents(docs, embedding_function)

    return db

In [12]:
def main():
    """Runs the module to build the embeddings and persists them in local"""

    print(f"Building embeddings for local")

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as exc:
        result_list = exc.map(lambda doc: get_documents(doc), data_path)

    final_docs_to_index = []

    for result in result_list:
        final_docs_to_index.extend(result)

    retriever = init_retriever(final_docs_to_index,embedding_function)

    return retriever

In [13]:
retriever = main()

Building embeddings for local
Extraction started for: ./data_3/Doc_1.pdf ...
Extraction started for: ./data_3/Doc_2.pdf ...
Extraction started for: ./data_3/Doc_3.pdf ...
Extraction started for: ./data_3/Doc_4.pdf ...
Extraction done for: ./data_3/Doc_3.pdf ...
Extraction started for: ./data_3/Doc_5.pdf ...
Extraction done for: ./data_3/Doc_4.pdf ...
Extraction done for: ./data_3/Doc_1.pdf ...
Extraction done for: ./data_3/Doc_5.pdf ...
Extraction done for: ./data_3/Doc_2.pdf ...


This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


In [14]:
query = "what are blockchain's impact on accounting and auditing practices ?"
docs = retriever.similarity_search(query,k=3)
docs

[Document(metadata={'source': 'Doc_1.pdf', 'title': 'G check for updates'}, page_content='G check for updates: Citation: Sheela, S.; Alsmady, A.A.; Tanaraj, K.; Izani, I. Navigating the Future: Blockchain’s Impact on Accounting and Auditing Practices. Sustainability 2023, 15, 16887. https://doi.org/10.3390/ su152416887Abstract: This study seeks to meticulously analyze the scholarly discussion on the integration of blockchain technology into accounting and auditing. Based on a total of 67 articles from the Web of Science (WoS) database, this study adopts a bibliometrics and content analysis approach which uses both numerical and visualization techniques to examine the extant literature. It spans the timeframe between 2016 and 2022. Bibliometrix R-package (Biblioshiny, version 4 is employed to analyze the descriptive analysis, which includes publication trends, the most trustworthy sources of scientiﬁc pub- lications, prominent scientiﬁc authors, prominent documents, and country collabor

### **Retrieval**

In [137]:
chat_llm = ChatOllama(
    model="phi3", # using phi3 open source model
    temperature=0,
    num_predict = 5000
)
chat_llm

ChatOllama(model='phi3', num_predict=5000, temperature=0.0)

In [16]:
def rerank_retrieved_docs(query, documents, top_k=5):
    docs_list = []

    for doc in documents:
        docs_list.extend([{"text": doc.page_content,
                           "meta": {"source": doc.metadata["source"]}}])

    ranker = Ranker(model_name="rank-T5-flan", cache_dir="./")

    rerank_request = RerankRequest(query=query, passages=docs_list)
    ranked_results = ranker.rerank(rerank_request)

    df = pd.DataFrame(ranked_results)
    df.sort_values(by="score", inplace=True, ascending=False)

    df = df.iloc[:top_k, :]  # pick top 5 by ranker score

    final_result = []

    for i in df.itertuples():
        final_result.append(Document(page_content=list(i)[1], metadata=list(i)[2]))

    return final_result

In [147]:
def retriever_tool(query):
    """This tool can answer questions related to Blockchain and cryptocurrency  """

    template = """You are a question/answering bot. Your job is to answer user question solely based on below context provided to you.
    Be as detailed and creative as you can be but make sure to only include the information from context in your answer.
    
    context:
    {context}

    Question: {question}
    Answer:

    Always Respond in valid JSON format with 'Answer' as key."Answer"'s value should be a valid string.If not, convert to string. Make sure you dont miss " while responding as json."""

    unique_docs_ = retriever.similarity_search(f"Represent this sentence for searching relevant passages: {query}")

    unique_docs = rerank_retrieved_docs(query=query, documents=unique_docs_,top_k=2)

    try:
        page_content = [d.page_content.strip() for d in unique_docs]
    except Exception as e:
        page_content= [None]

    prompt = PromptTemplate(template=template,
                            input_variables=["context", "question"])

    qa_chain = LLMChain(llm=chat_llm, prompt=prompt)

    llm_op = qa_chain(
        inputs={
            "question": query,
            "context": "\n---\n".join(page_content)[:5000]  # only send 5000 character as context
        })

    try:
        op_src_temp = [src.metadata.get("source") for src in unique_docs]
        op_src = list(set([f"{x.split('/')[-1]}" for x in op_src_temp]))
    except Exception as e:
        op_src = "/Not found"
    print(llm_op)
    try:
        llm_op_json = json.loads(llm_op['text'].replace("json", "").replace("```", "").replace("\n", ""))
    except Exception as e:
        llm_op_json = json.loads(llm_op['text']+"\"")

    if isinstance(op_src, str):
        f_name = op_src.split("/")[-1]
        response = {"message": llm_op_json["Answer"],
                    "metadata": {"source": [f_name]},
                    "retrieved_docs": page_content
                    }
    else:
        response = {"message": llm_op_json["Answer"],
                    "metadata": {"source": op_src} ,
                    "retrieved_docs": page_content
                    }

    return response


In [26]:
answer = retriever_tool(query=query)
answer

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'message': "# Blockchain Impact on Accounting and Auditing Practices\n\nBlockchain technology has the potential to significantly transform accounting and auditing practices by offering features such as transparency, immutability, security, and decentralization. The integration of triple-entry accounting can enhance precision, reliability, and transparency in financial information while reducing risks associated with malfeasance and increasing stakeholder confidence.\n\nKey impacts include:\n\n1. **Automation**: Blockchain technology automates repetitive tasks within the auditing process by providing real-time access to financial data, which streamlines workflows, reduces costs, time spent on manual processes, and ensures tamper-proof evidence.\n   \n2. **New Business Models**: The adoption of blockchain can lead to new business models such as decentralized autonomous organizations (DAOs), reducing intermediaries in transactions which results in cost reduction, increased efficiency, an

In [27]:
answer["message"]

"# Blockchain Impact on Accounting and Auditing Practices\n\nBlockchain technology has the potential to significantly transform accounting and auditing practices by offering features such as transparency, immutability, security, and decentralization. The integration of triple-entry accounting can enhance precision, reliability, and transparency in financial information while reducing risks associated with malfeasance and increasing stakeholder confidence.\n\nKey impacts include:\n\n1. **Automation**: Blockchain technology automates repetitive tasks within the auditing process by providing real-time access to financial data, which streamlines workflows, reduces costs, time spent on manual processes, and ensures tamper-proof evidence.\n   \n2. **New Business Models**: The adoption of blockchain can lead to new business models such as decentralized autonomous organizations (DAOs), reducing intermediaries in transactions which results in cost reduction, increased efficiency, and improved t

In [28]:
answer["metadata"]

{'source': ['Doc_1.pdf']}

### **Evaluation**

In [132]:
# Question and their correct answer pair manually created from all 5 docs.

queries = {"what are some potential future research areas related to the regulatory and governance aspects of blockchain technology in accounting and auditing":
'''Governance Frameworks: Future research could focus on developing governance frameworks tailored to the decentralized nature of blockchain. This includes studying current governance structures and proposing new frameworks that align with the unique characteristics of blockchain technology in accounting and auditing.
Regulatory Guidelines: Research is needed to explore regulatory guidelines for blockchain application in these fields. This involves investigating existing legal frameworks to identify gaps and determining how they can be adapted to ensure compliance while fostering innovation.
Stakeholder Perspectives: Understanding the perspectives of various stakeholders, such as regulators, industry professionals, and blockchain users, is crucial. Surveys and interviews could provide insights into the expectations and challenges related to regulatory compliance.
Balancing Innovation and Security: Research could examine how to balance the potential of blockchain technology for innovation with the need for security and regulatory compliance. This might include studying the impact of blockchain on existing compliance measures and suggesting improvements. ''',  
    
    "What are the potential benefits of implementing blockchain technology in e-commerce transactions":
        ''' Enhanced Security: Blockchain technology offers a decentralized and tamper-resistant system, ensuring the security of e-commerce transactions by making them immutable and transparent.
Fraud Prevention: By using cryptographic techniques and decentralized identity management, blockchain helps in reducing fraud risks and safeguarding user identities.
Transparent Transactions: The transparency of blockchain creates a detailed and immutable record of transactions, aiding in fraud detection and enhancing trust.
Supply Chain Transparency: Blockchain improves supply chain management by providing a transparent and unalterable audit trail, ensuring product authenticity and efficient tracking.
Smart Contracts: These automate transactions based on predefined rules, minimizing potential disputes and eliminating the need for intermediaries in contract enforcement.
Decentralized Marketplaces: Blockchain enables secure peer-to-peer transactions, reducing the reliance on centralized authorities and enhancing marketplace security.''',

"What are some of the challenges faced by traditional financial institutions like Barclays and Citigroup when integrating cryptocurrencies into their services?":
'''Traditional financial institutions such as Barclays and Citigroup face several challenges when integrating cryptocurrencies into their services. Barclays collaborates with fintech
 companies to offer crypto services, which exposes them to challenges like maintaining regulatory compliance. Working with third parties can complicate the bank's commitment to data
  security and customer service. Meanwhile, Citigroup’s approach involves developing its own crypto trading platform. This strategy presents challenges such as ensuring cybersecurity to protect
   customers’ digital assets and sensitive information, highlighting the need for robust cybersecurity measures to mitigate risks like hacking and fraud. These institutions must balance 
   innovative services with the responsibility of adhering to regulatory standards and ensuring the security of their systems and customer data. ''',

"What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies":
''' Bitcoin: The generalized hyperbolic distribution provides the best fit for Bitcoin's log returns.
Litecoin: Similar to Bitcoin, the generalized hyperbolic distribution gives the best fit for Litecoin.
Dash, Monero, and Ripple: For these cryptocurrencies, the normal inverse Gaussian distribution is the best-fitting distribution.
Dogecoin: The generalized t distribution provides the best fit for Dogecoin.
MaidSafeCoin: The Laplace distribution gives the best fit for MaidSafeCoin.
These results indicate that no single distribution fits all cryptocurrencies' log returns; instead, different distributions are best suited to different cryptocurrencies, highlighting the 
unique statistical properties of each cryptocurrency's exchange rate behavior '''
,
"What are the potential disadvantages of cryptocurrencies from market analysis perspective":
''' Volatility and Speculation: The market for cryptocurrencies is highly volatile, and their value is often driven by speculation rather than a backing asset. This volatility raises concerns about speculative bubbles and the risk of significant price fluctuations.
Regulatory Concerns: The decentralized and largely unregulated nature of cryptocurrencies raises concerns about regulatory evasion, money laundering, and their use in illegal activities. This has led to calls for increased regulation of the cryptocurrency market.
Energy Consumption: The process of "mining" cryptocurrencies requires substantial computational efforts, leading to high energy consumption and raising sustainability concerns. The environmental impact of cryptocurrency mining is significant and often criticized.
Security Risks: Although the technology provides security for transactions, cryptocurrencies are not immune to scams, frauds, and security breaches, such as the hacking of exchanges and theft of funds. '''

        }


In [133]:
def test_case(queries,set_threshold):

    MODEL = "gpt-4-32k" # using external model for evaluation.

    contextual_precision = ContextualPrecisionMetric(threshold=set_threshold) # Define in environment variable. threshold
    contextual_recall = ContextualRecallMetric(threshold=set_threshold)
    contextual_relevancy = ContextualRelevancyMetric(threshold=set_threshold)
    answer_relevancy = AnswerRelevancyMetric(threshold=set_threshold)
    faithfulness = FaithfulnessMetric(threshold=set_threshold)
    hallucination = HallucinationMetric(threshold=set_threshold)
    
    all_test_cases = []

    for query,actual_answer in queries.items():
        print("Evaluation running...")
        llm_response = retriever_tool(query=query)
    
        test_case = LLMTestCase(
            input=query,
            actual_output=llm_response["message"],
            expected_output=actual_answer,
            retrieval_context=llm_response["retrieved_docs"],
            context=llm_response["retrieved_docs"]
        )

        metrics=[contextual_precision, contextual_recall, contextual_relevancy, answer_relevancy, faithfulness, hallucination]

        all_test_cases.append(test_case)

    if all_test_cases:
        ans = evaluate(
            test_cases=all_test_cases,
            metrics = metrics
            
        )
    else:
        print("no test cases ")
        pass
    return ans

In [148]:
result = test_case(queries=queries, set_threshold=0.7)

Evaluation running...
{'question': 'what are some potential future research areas related to the regulatory and governance aspects of blockchain technology in accounting and auditing', 'context': 'G check for updates: Citation: Sheela, S.; Alsmady, A.A.; Tanaraj, K.; Izani, I. Navigating the Future: Blockchain’s Impact on Accounting and Auditing Practices. Sustainability 2023, 15, 16887. https://doi.org/10.3390/ su152416887Abstract: This study seeks to meticulously analyze the scholarly discussion on the integration of blockchain technology into accounting and auditing. Based on a total of 67 articles from the Web of Science (WoS) database, this study adopts a bibliometrics and content analysis approach which uses both numerical and visualization techniques to examine the extant literature. It spans the timeframe between 2016 and 2022. Bibliometrix R-package (Biblioshiny, version 4 is employed to analyze the descriptive analysis, which includes publication trends, the most trustworthy 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


✨ You're running DeepEval's latest Contextual Precision Metric! (using azure openai, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using azure openai, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using azure openai, strict=False, 
async_mode=True)...

...

✨ You're running DeepEval's latest Faithfulness Metric! (using azure openai, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Hallucination Metric! (using azure openai, strict=False, async_mode=True)...

Evaluating 5 test case(s) in parallel: |          |  0% (0/5) [Time Taken: 00:00, ?test case/s]INFO:httpx:HTTP Request: POST https://openaicoenorthcentralus.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openaicoenorthcentralus.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 9.000000 seconds
INFO:httpx:HTTP Request: POST https://openaicoenorthcentralus.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openaicoenorthcentralus.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openaicoenorthcentralus.openai.azure.com/openai/deployments/gpt-4-32k/cha



Metrics Summary

  - ✅ Contextual Precision (score: 1.0, threshold: 0.7, strict: False, evaluation model: azure openai, reason: The score is 1.00 because all nodes in the retrieval context provide relevant and useful information to the input query. Each node either directly or indirectly provides insights into the best-fitting statistical distributions for the log returns of major cryptocurrencies., error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: azure openai, reason: The score is 1.00 because all sentences in the expected output are well supported by information in the 1st node in the retrieval context. This includes the best fitting distributions for various cryptocurrencies and the observation that no single distribution fits all cryptocurrencies' log returns. Great job!, error: None)
  - ✅ Contextual Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: azure openai, reason: The score is 1.00 because the retriev

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [149]:
result_list = []

for j in range(5):
    for i in range(6):
        result_list.append([
        result.test_results[j].name,
        result.test_results[j].input,
        result.test_results[j].metrics_data[i].name,
        result.test_results[j].metrics_data[i].score,
        result.test_results[j].metrics_data[i].success,
        result.test_results[j].metrics_data[i].reason])

In [150]:
pd.set_option("max_colwidth", None)
result_df = pd.DataFrame(result_list,columns=["TC_Num.","Query","Metrics","Score","Pass/Fail","Reason"])
result_df["Pass/Fail"] = result_df["Pass/Fail"].map({True:'Pass',False:"Fail"})

In [151]:
result_df

,TC_Num.,Query,Metrics,Score,Pass/Fail,Reason
0,test_case_3,What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies,Contextual Precision,1.000000,Pass,The score is 1.00 because all nodes in the retrieval context provide relevant and useful information to the input query. Each node either directly or indirectly provides insights into the best-fitting statistical distributions for the log returns of major cryptocurrencies.
1,test_case_3,What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies,Contextual Recall,1.000000,Pass,The score is 1.00 because all sentences in the expected output are well supported by information in the 1st node in the retrieval context. This includes the best fitting distributions for various cryptocurrencies and the observation that no single distribution fits all cryptocurrencies' log returns. Great job!
2,test_case_3,What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies,Contextual Relevancy,1.000000,Pass,"The score is 1.00 because the retrieval context directly addresses the input question about the best-fitting statistical distributions for the log returns of exchange rates of major cryptocurrencies. It provides detailed information about the analysis of the largest cryptocurrencies, the statistical properties of their exchange rates, and the specific distributions that fit best for different cryptocurrencies."
3,test_case_3,What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies,Answer Relevancy,1.000000,Pass,The score is 1.00 because the answer accurately and thoroughly responds to the query about the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies without including any irrelevant information.
4,test_case_3,What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies,Faithfulness,1.000000,Pass,The score is 1.00 because there are no contradictions noted between the actual output and the retrieval context.
5,test_case_3,What are the best-fitting statistical distributions for the log returns of exchange rates of the major cryptocurrencies,Hallucination,0.500000,Pass,"The score is 0.50 because while the output aligns with the first context about the generalized hyperbolic distribution fitting best for Bitcoin and Litecoin, it contradicts the second context by not discussing the visual representation of the distribution of the log returns of the seven cryptocurrencies."
6,test_case_2,What are some of the challenges faced by traditional financial institutions like Barclays and Citigroup when integrating cryptocurrencies into their services?,Contextual Precision,1.000000,Pass,The score is 1.00 because all nodes in the retrieval context provide relevant and useful information about the challenges faced by traditional financial institutions like Barclays and Citigroup when integrating cryptocurrencies into their services. Great job!
7,test_case_2,What are some of the challenges faced by traditional financial institutions like Barclays and Citigroup when integrating cryptocurrencies into their services?,Contextual Recall,1.000000,Pass,"The score is 1.00 because all sentences in the expected output were fully attributed to the 2nd node in the retrieval context, indicating a perfect match between the generated output and the context provided."
8,test_case_2,What are some of the challenges faced by traditional financial institutions like Barclays and Citigroup when integrating cryptocurrencies into their services?,Contextual Relevancy,0.928571,Pass,"The score is 0.93 because while the retrieval context contains detailed information about the challenges and strategies of integrating cryptocurrencies for traditional financial institutions like Ba

In [152]:
result_df.to_csv("./simple_RAG.csv")